In [ ]:
from shared_code.utility.schemas.spark_table_schema import image_table_schema,image_table_schema_with_caption_tokens
from shared_code.utility.spark.set_environ import *
from shared_code.utility.storage.blob import BlobAdapter
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

set_azure_env()

spark = get_session('cloud-mover')

In [ ]:
spark.read.parquet("D:\\data\\processed\\reddit_images_processed_with_caption_tokens.parquet", schema=image_table_schema_with_caption_tokens).createOrReplaceTempView("reddit_images_processed_with_caption_tokens")

existing_images_not_curated = spark.sql("""
select * from
reddit_images_processed_with_caption_tokens
where small_image is not null
and small_image != ''
and small_image != 'None'
and small_image != 'Nan'
and length(updated_caption)  > 5
and length(caption) > 5
and updated_caption not like '%Error%'
and exists == True
and not curated
""")

print(f"Total images (Not Curated): {existing_images_not_curated.count()}")

existing_images_not_curated.limit(10).toPandas()

In [ ]:
foo = existing_images_not_curated.rdd.map(lambda x: ({'image_path': x['small_image'], 'name': x['image_name'], 'sub':x['subreddit'], 'caption': x['updated_caption'], 'text': x['text']})).collect()

In [ ]:
blob_container_name = 'unprocessed'
blob_adapter = BlobAdapter(blob_container_name)

container_client = blob_adapter.get_container_client()

total_items = len(foo)
print(f"Total items: {total_items}")
for elem in foo:
    if total_items % 100 == 0:
        print(f"Remaining: {total_items}")
    data = elem['image_path']
    with open(data, "rb") as f:
        data = f.read()
        out_path = f"{elem['sub']}/{elem['name']}"
        blob_adapter.upload_blob(data=data, blob_name=out_path)
        total_items -= 1

In [ ]:
foo.to_csv("D:\\data\\processed\\manifest.csv", index=False)
with open("D:\\data\\processed\\manifest.csv", "rb") as f:
    data = f.read()
    blob_adapter.upload_blob(data=data, blob_name="manifest.csv")